In [53]:
from Bio import Entrez
from xml.etree import ElementTree as ET
import re
import requests
Entrez.email = "asamatt@umich.edu"

In [66]:
handle = Entrez.esearch(db="pubmed", retmax=10, term="Brucella vaccine")
record = Entrez.read(handle)
print(record)

{'Count': '2294', 'RetMax': '10', 'RetStart': '0', 'IdList': ['40800113', '40791355', '40788720', '40739071', '40733172', '40715363', '40711265', '40665323', '40645225', '40640584'], 'TranslationSet': [{'From': 'Brucella vaccine', 'To': '"brucella vaccine"[Supplementary Concept] OR "brucella vaccine"[All Fields] OR "brucella vaccine"[MeSH Terms] OR ("brucella"[All Fields] AND "vaccine"[All Fields])'}], 'QueryTranslation': '"brucella vaccine"[Supplementary Concept] OR "brucella vaccine"[All Fields] OR "brucella vaccine"[MeSH Terms] OR ("brucella"[All Fields] AND "vaccine"[All Fields])'}


In [ ]:
def convert_pmid_to_pmcid(pmid):
  url = f"https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?ids={pmid}&format=json"
  response = requests.get(url)
  data = response.json()
  if "records" in data and data["records"]:
    return data["records"][0].get("pmcid", None)
  return None

In [ ]:
print(type(record))
if isinstance(record, dict):
	print(record.keys())
	print(record['IdList'])
	abstracts = []
	for id in record['IdList']:
		pmcid = convert_pmid_to_pmcid(id)
		print(pmcid)
		if pmcid:
			pmcid_handle = Entrez.efetch(db="pmc", id=pmcid, rettype="abstract", retmode="text")
			abstract = pmcid_handle.read()
			print(abstract)
			abstracts.append(abstract)
   

In [59]:
def get_element_text(elem):
    """Recursively extract all text from an ElementTree element."""
    text_parts = []
    if elem.text:
        text_parts.append(elem.text)
    for child in elem:
        text_parts.append(get_element_text(child))
        if child.tail:
            text_parts.append(child.tail)
    return "".join(text_parts)

In [63]:
print(type(record))
if isinstance(record, dict):
	print(record.keys())
	print(record['IdList'])
	abstracts = []
	for id in record['IdList']:
		pmid_handle = Entrez.efetch(db="pubmed", id=id, rettype="abstract", retmode="xml")
		xml_data = pmid_handle.read().decode("utf-8")
		root = ET.fromstring(xml_data)
		for abstract_text in root.findall(".//Abstract"):
			abstracts.append({"pmid": id, "abstract": get_element_text(abstract_text)})

<class 'Bio.Entrez.Parser.DictionaryElement'>
dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'QueryTranslation'])
['40800113', '40791355', '40788720', '40739071', '40733172', '40715363', '40711265', '40665323', '40645225', '40640584']


In [64]:
for abstract in abstracts:
  print(abstract)
  print("\n")

{'pmid': '40800113', 'abstract': 'Brucella bacteria are adept at evading the human immune system, leading to a chronic infectious disease known as brucellosis, which poses significant global health challenges. This study addresses a notable gap in bibliometric analyses concerning the immune response to brucellosis.We conducted a comprehensive literature screening from the Web of Science Core Collection, covering publications from 1980 to 2024. Relevant publications were analyzed using R software, VOSviewer, and CiteSpace for bibliometric analysis.A total of 733 publications were included in this study, revealing an average annual growth rate of 5.91% in publications. The United States led in publication volume and citations, followed by China and France. Prominent research institutions included INSERM (France) and CONICET (Argentina). Infection and Immunity was identified as a leading journal in the field, publishing 97 papers with 5,184 citations. Keyword co-occurrence analysis deline

In [ ]:
print(abstracts[0])
clean_abstract = re.search(r"<abstract.*?>.*?</abstract>", abstracts[0], re.DOTALL)

In [ ]:
print(clean_abstract.group(0))